In [37]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from secrets import IEX_CLOUD_API_TOKEN

In [38]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [39]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 84746334, 'calculationPrice': 'close', 'change': -5.17, 'changePercent': -0.03059, 'close': 169.08, 'closeSource': 'iofaflic', 'closeTime': 1669626135285, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 166.1, 'delayedPriceTime': 1727447241007, 'extendedChange': -0.24, 'extendedChangePercent': -0.00142, 'extendedPrice': 170.66, 'extendedPriceTime': 1682286012994, 'high': 173.5, 'highSource': 'olesC', 'highTime': 1719707891451, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 172.32, 'iexCloseTime': 1704520514497, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 173.04, 'iexOpenTime': 1656910012942, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1697059069540, 'latestPrice': 165.52, 'latestSource': 'Close', 'latestTime': 'April 14, 2022', 'latestUpdate': 1662367559687, 'latestVolume': 75926558, 'low': 169.08, 'lowSource': 'soelC', 'lowTime': 1722736414264, '

In [40]:
price = data['latestPrice']
market_cap = data['marketCap']

In [41]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [42]:
symbol_groups = list([stocks['Ticker'][x:x+100] for x in range(0, len(stocks['Ticker']), 100)])
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])    

final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns
            ),
            ignore_index=True
        )
        except KeyError as e:
            pass
        
final_dataframe

/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_28406/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_28406/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_28406/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_28406/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,126.84,38554461364,N/A
1,AAL,19,12610713482,N/A
2,AAP,227.45,13551933453,N/A
3,AAPL,170.59,2741230875343,N/A
4,ABBV,163.25,294851419173,N/A
...,...,...,...,...
496,YUM,128.25,36308695975,N/A
497,ZBH,132,26421470702,N/A
498,ZBRA,394.88,21264942077,N/A
499,ZION,65.3,9983661716,N/A


In [43]:
# Calculating number of shares to buy
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    portfolio_size = input('Please enter an integer')
    val = float(portfolio_size)

In [44]:
position_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,126.84,38554461364,15
1,AAL,19,12610713482,105
2,AAP,227.45,13551933453,8
3,AAPL,170.59,2741230875343,11
4,ABBV,163.25,294851419173,12
...,...,...,...,...
496,YUM,128.25,36308695975,15
497,ZBH,132,26421470702,15
498,ZBRA,394.88,21264942077,5
499,ZION,65.3,9983661716,30


In [45]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)


In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [ ]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].write(f'{column}1', f'{column_formats[column][0]}', column_formats[column][1])
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    
writer.save()